In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO

seqs = []

for record in SeqIO.parse('uorfs_7_64.fasta','fasta'):
    seqs.append(str(record.seq))

In [2]:
print(len(seqs))

402140


In [3]:
# read natural:
df = pd.read_csv('./UTRGAN/data/utrdb2.csv')
df = df['seq'].to_numpy()
nats = []
for i in range(len(df)):
    if  len(df[i])< 129 and len(df[i]) > 64:
        nats.append(df[i].replace('T','U'))

In [4]:
# read generated
inits = []
for record in SeqIO.parse('/data4/sina/UTR/MEME/1024_10000_init.fasta','fasta'):
    inits.append(str(record.seq))

opts = []
for record in SeqIO.parse('/data4/sina/UTR/MEME/1024_10000_opt.fasta','fasta'):
    opts.append(str(record.seq))

In [5]:
# read optimus
optimus = []
for record in SeqIO.parse('/data4/sina/UTR/MEME/optimus_fasta.fasta','fasta'):
    optimus.append(str(record.seq))

In [6]:
path = '/data4/sina/UTR/UTRGAN/src/mrl_te_optimization/outputs/opt_10000_1024_saved/init_seqs_FMRL_10000.txt'
with open(path, 'r') as f:
    lines = f.readlines()

mut_inits = []
for i in range(len(lines)):
    mut_inits.append(lines[i].replace('\n','')[:50])



In [7]:
nat_trun = []
init_trun = []
opt_trun = []
m_trun = []



with open('natural_truncated.fasta','w') as f:
    for i in range(len(nats)):
        f.write(f'>seq_{i}\n')
        nat_trun.append(nats[i][:50])
        f.write(str(nats[i][:50])+'\n')

In [11]:
def transform(seq:str):
    seq = seq.lower()
    seq = seq.replace('ggg','p')
    while 'pg' in seq:
        seq = seq.replace('pg','p')

    while 'gp' in seq:
        seq = seq.replace('gp','p')
    
    return seq

# find all g-runs
def has_gquad(seq):
    seq = transform(seq)
    print(seq)
    if seq.count('p') < 4:
        return False
    else:
        occurances = []
        for i in range(len(seq)):
            if seq[i] == 'p':
                occurances.append(i)
        dists = []
        for i in range(len(occurances)-1):
            dists.append(occurances[i+1] - occurances[i])

        start = False
        count = 0
        if len(dists) > 0:
            for i in range(len(dists)):
                if dists[i] <= 232:
                    start = True
                    count += 1
                else:
                    count = 0
                    start = False
        else:
            return False


        if start and count >= 4:

            return True

        else: 

            return False 

In [ ]:
print(len(inits))

In [12]:
import time

init_g = 0
opt_g = 0
optimus_g = 0
nat_g = 0

start = time.time()

t = has_gquad(inits[0])

end = time.time()

print(end-start)

print("starting inits")

for i in range(len(inits)):
    if has_gquad(inits[i]):
        init_g += 1

print("starting opts")


for i in range(len(opts)):
    if has_gquad(opts[i]):
        opt_g += 1

print("starting optimus")

for i in range(len(optimus)):
    if has_gquad(optimus[i]):
        opt_g += 1

print("starting nats")

for i in range(len(nats)):
    if has_gquad(nats[i]):
        nat_g += 1


augucgaugauccggagaagacaacugugaccucaaguugucupagcuucupacuuggaggcugucccgcugagucag
0.00033855438232421875
starting inits
...
starting opts
...
starting optimus
...
starting nats
...


In [14]:
print(init_g)
print(opt_g)
print(nat_g/32000.*1024)
print(optimus_g)

98
28
89.92
0


In [43]:
init_c = 0
opt_c = 0
optimus_c = 0
nat_c = 0

for i in range(len(seqs)):
    if i != 0 and i % 1000 == 0:
        print(f"checkpoint: {i}")
    for j in range(len(nats)):
        if seqs[i] in nats[j]:
            nat_c += 1
    for j in range(len(inits)):
        if seqs[i] in inits[j]:
            init_c += 1
    for j in range(len(opts)):
        if seqs[i] in opts[j]:
            opt_c += 1
    for j in range(len(optimus)):
        if seqs[i] in optimus[j]:
            optimus_c += 1

checkpoint: 1000
checkpoint: 2000
checkpoint: 3000
checkpoint: 4000
checkpoint: 5000
checkpoint: 6000
checkpoint: 7000
checkpoint: 8000
checkpoint: 9000
checkpoint: 10000
checkpoint: 11000
checkpoint: 12000
checkpoint: 13000
checkpoint: 14000
checkpoint: 15000
checkpoint: 16000
checkpoint: 17000
checkpoint: 18000
checkpoint: 19000
checkpoint: 20000
checkpoint: 21000
checkpoint: 22000
checkpoint: 23000
checkpoint: 24000
checkpoint: 25000
checkpoint: 26000
checkpoint: 27000
checkpoint: 28000
checkpoint: 29000
checkpoint: 30000
checkpoint: 31000
checkpoint: 32000
checkpoint: 33000
checkpoint: 34000
checkpoint: 35000
checkpoint: 36000
checkpoint: 37000
checkpoint: 38000
checkpoint: 39000
checkpoint: 40000
checkpoint: 41000
checkpoint: 42000
checkpoint: 43000
checkpoint: 44000
checkpoint: 45000
checkpoint: 46000
checkpoint: 47000
checkpoint: 48000
checkpoint: 49000
checkpoint: 50000
checkpoint: 51000
checkpoint: 52000
checkpoint: 53000
checkpoint: 54000
checkpoint: 55000
checkpoint: 56000
c

In [41]:
kozaks = ['AAAAAAAUG','GCCGCCAUG','ACAGCAAUG','AACACCAUG','GAAGCAAUG','GCAACAAUG',
          'ACAACCAUG','ACAGACAUG','GCACAAAUG','GCAAACAUG','GACAACAUG','GACACAAUG',
          'GAAACCAUG','ACCACAAUG','GAAGACAUG','ACCAACAUG','AACGCAAUG','GACGAAAUG','ACCGAAAUG']
starts = ['AUG','GUG','CUG','AUA']
# starts = ['AUG']

kozak_t = []
for i in range(len(kozaks)):
    for j in range(len(starts)):
        kozak_t.append(kozaks[i][:-3]+starts[j])


In [42]:
print(kozak_t[0])

AAAAAAAUG


In [22]:
print(nats[4])

AGACAAGGUUUCCCUCUGUAACCCAGGCUGGAGUGCAGUGGAACUUCUGAGCUCAAGUAGUCUUCACAUCUCAGCCUUCCAAGUAGCUGUGACUAGAG


In [38]:
nat_k = 0
init_k = 0
opt_k = 0
optimus_k = 0

kozaks = kozak_t
# kozaks = [starts[0]]

for i in range(len(kozaks)):
    if i != 0 and i % 1000 == 0:
        print(f"checkpoint: {i}")
    for j in range(len(nats)):
        if kozaks[i] in nats[j]:
            nat_k += 1
    for j in range(len(inits)):
        if kozaks[i] in inits[j]:
            init_k += 1
    for j in range(len(opts)):
        if kozaks[i] in opts[j]:
            opt_k += 1
    for j in range(len(optimus)):
        if kozaks[i] in optimus[j]:
            optimus_k += 1


In [24]:
nat_pos = []
init_pos = []
opt_pos = []
optimus_pos = []

kozaks = kozaks

for i in range(len(kozaks)):
    if i != 0 and i % 1000 == 0:
        print(f"checkpoint: {i}")
    for j in range(len(nats)):
        if kozaks[i] in nats[j]:
            t = nats[j].find(kozaks[i])
            nat_pos.append(len(nats[j])-t)
    for j in range(len(inits)):
        if kozaks[i] in inits[j]:
            t = inits[j].find(kozaks[i])
            init_pos.append(len(inits[j])-t)
    for j in range(len(opts)):
        if kozaks[i] in opts[j]:
            t = opts[j].find(kozaks[i])
            opt_pos.append(len(opts[j])-t)
    for j in range(len(optimus)):
        if kozaks[i] in optimus[j]:
            t = optimus[j].find(kozaks[i])
            optimus_pos.append(len(optimus[j])-t)

In [36]:
print(np.average(nat_pos))
print(np.average(init_pos))
print(np.average(opt_pos))
print(np.average(optimus_pos))

55.89917936694021
60.76
48.38095238095238
17.333333333333332


In [39]:
print(nat_k/32000.*1024)
print(opt_k)
print(init_k)
print(optimus_k)

27.296
42
25
9
